# Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [76]:
import pandas as pd

In [62]:
data = pd.read_csv('ratings.csv')

In [68]:
# Решение в одну строчку (самое быстрое)
data['class'] = data['rating'].apply(lambda x: 'низкий рейтинг' if x <= 2 else ('средний рейтинг' if x <= 4 else 'высокий рейтинг'))
data.head(10)
# data.head()[['movieId','class']] - можно выбрать конкретные столбцы, которые хочешь увидеть

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг
5,1,1263,2.0,1260759151,низкий рейтинг
6,1,1287,2.0,1260759187,низкий рейтинг
7,1,1293,2.0,1260759148,низкий рейтинг
8,1,1339,3.5,1260759125,средний рейтинг
9,1,1343,2.0,1260759131,низкий рейтинг


In [64]:
data['class'].value_counts()

средний рейтинг    63801
высокий рейтинг    22818
низкий рейтинг     13385
Name: class, dtype: int64

In [7]:
# 2 вариант решения
data_1 = pd.read_csv('ratings.csv')
data_1.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
# 2 вариант решения, через функцию 
def class_(rt):
    if rt <= 2:
        return 'низкий рейтинг'
    elif rt <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'

In [9]:
data['class'] = data['rating'].apply(class_)
data.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


# Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [77]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}


In [78]:
data_1 = pd.read_csv('keywords.csv')

In [111]:
# Эксперимент
# чтобы ускорить обработку запроса можно сделал список городов через множетсво,попытался применить, НО не помогло, хотя если я правильно понимаю эффект должен был быть 
new_geo_data = set()
for region in geo_data:
    for city in geo_data[region]:
        new_geo_data.add(city)

In [124]:
def find(row):
    for city in new_geo_data:
        if city in row['keyword']:
            return city
    return 'undefined'

In [127]:
%%time
data_1['city'] = data_1.apply(find, axis = 1)

Wall time: 14.7 s


In [114]:
#Основное Решение задачи
def find_region(row):
    for region in geo_data:
        for city in geo_data[region]:
            if city in row['keyword']:
                return region
    return 'undefined'

In [123]:
%%time
data_1['region'] = data_1.apply(find_region, axis = 1)

Wall time: 15.3 s


In [109]:
data_1['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

# Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [34]:
year = range(1950,2011)

In [35]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [36]:
def production_year(title):
    for element in year:
        if str(element) in title:
            return element
    return '1900'

In [38]:
movies['year'] = movies['title'].apply(production_year)
movies.head(100)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
95,103,Unforgettable (1996),Mystery|Sci-Fi|Thriller,1996
96,104,Happy Gilmore (1996),Comedy,1996
97,105,"Bridges of Madison County, The (1995)",Drama|Romance,1995
98,107,Muppet Treasure Island (1996),Adventure|Children|Comedy|Musical,1996


In [39]:
merged_movies = pd.merge(left = data, right = movies, left_on = 'movieId', right_on = 'movieId')
merged_movies.head()

,userId,movieId,rating,timestamp,class,title,genres,year
0,1,31,2.5,1260759144,средний рейтинг,Dangerous Minds (1995),Drama,1995
1,7,31,3.0,851868750,средний рейтинг,Dangerous Minds (1995),Drama,1995
2,31,31,4.0,1273541953,средний рейтинг,Dangerous Minds (1995),Drama,1995
3,32,31,4.0,834828440,средний рейтинг,Dangerous Minds (1995),Drama,1995
4,36,31,3.0,847057202,средний рейтинг,Dangerous Minds (1995),Drama,1995


In [40]:
obj = merged_movies.groupby('year') ['rating']
obj.mean().sort_values(ascending = False).head(20)

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
1974    3.978704
1962    3.952446
1950    3.915254
1977    3.905786
1964    3.841492
1959    3.841033
1958    3.823171
1975    3.823077
1969    3.817376
1967    3.814516
1960    3.809211
1976    3.806794
1973    3.806667
1979    3.777966
1981    3.758763
Name: rating, dtype: float64

In [41]:
merged_movies['old'] = merged_movies['year'].apply(lambda year: int(year) < 1970)
obj = merged_movies.groupby('old') ['rating']
obj.mean().sort_values(ascending = False).head(20)

old
True     3.705525
False    3.519579
Name: rating, dtype: float64

# Старые фильмы - лучше новых!